In [ ]:
import sys
import os

In [ ]:
sys.path.append(os.getcwd())

In [ ]:
%matplotlib inline

from pycocotools.coco import COCO
from pycocoevalcap.eval import COCOEvalCap
import matplotlib.pyplot as plt
import skimage.io as io
import pylab
pylab.rcParams['figure.figsize'] = (10.0, 8.0)

import json
from json import encoder
encoder.FLOAT_REPR = lambda o: format(o, '.3f')

from random import randint, uniform,random
import pandas as pd
import numpy as np


from PIL import Image
from IPython.core.display import HTML

from datetime import date


In [ ]:
def check_len_cap(resFile, n_limit):
    
    with open(resFile) as f:
                preds = json.load(f)

    preds_df = pd.DataFrame(preds)

    indexes_long_cap = preds_df[preds_df['caption'].str.split().apply(len)>n_limit].index.values

    if(len(indexes_long_cap)>0):
        for i in (indexes_long_cap): preds_df.at[i, 'caption'] = ' '.join(preds_df.at[i, 'caption'].split()[:n_limit])

        preds_df.to_json(resFile[:-5]+'-CUT.json',orient='records')

        resFile = resFile[:-5]+'-CUT.json' 
        print('\nINFO: Algunos subtítulos han sido cortados.')

    else:
        print('\nINFO: Todos los subtítulos tiene el tamaño correcto.\n')
    return resFile


In [ ]:
# Ajustar path de imágenes
dataDir= '/'.join(map(str, os.getcwd().split("/")[0:-1])) 
dataType='val2014'
algName = 'fakecap'
annFile='%s/score/annotations/captions_%s.json'%(dataDir,dataType)

rootDir = "/".join(os.getcwd().split("/")[0:-1])+"/"

# Se debe especifcar la ruta del archivo de predicciones. 
resFile = rootDir+'transformer-tensorflow/output/transformer-limiter-tf-20210721-175529-predictions.json' # EJEMPLO


# Descargar modelos Stanford
!./get_stanford_models.sh

In [ ]:
ls /export/gts_usuarios/aghernandez/automatic-description-images/rnn-tensorflow/output/

In [ ]:
# [OPCIONAL]: Variables para guardar información
mode = ['tf-transformer/', 'fairseq/', 'tf-RNN/']
savedDir = 'saved/'+ mode[2]
savedFile="-".join(resFile.split('/')[-1].split('.')[0:-1])+"-"+str(date.today().strftime("%d%m%Y"))

In [ ]:
resFile = check_len_cap(resFile, n_limit=25)

In [ ]:
def image_path(coco_dir, image_id):
    return os.path.join(coco_dir, 'images', 'val2014', f'COCO_val2014_{image_id:012d}.jpg')

def image_size(image_path):
    with Image.open(image_path) as img:
        return img.size


def image_html(image_path, scale):
    width, height = image_size(image_path)
    return f'<img src="{image_path}" width="{width * scale}" height="{height * scale}" style="display: block; margin-left: auto; margin-right: auto;" />'


def result_html(coco_dir,
                image_ids,
                image_captions,
                score,
                image_scale=0.3):
    
    image_paths = [image_path(coco_dir, image_id) for image_id in image_ids]


    df = pd.DataFrame(dict({'ImageID': image_ids,
                            'Image': image_paths,
                            'Captions': image_captions,
                            'CIDEr Score': score['CIDEr'],
                            'SPICE Score': score['SPICE']
                            }))
    
    return df.to_html(index=False,
                      escape=False,
                      justify='left',
                      formatters=[lambda x: '<p style="text-align: center;">' + str(x) + '</p>', 
                                  lambda path: image_html(path, image_scale), 
                                  #lambda x: '<p style="text-align: center;">' + x + '</p>', 
                                  lambda x: '<p style="text-align: center;">' + x + '</p>',
                                  lambda x: '<p style="text-align: center;">' + x + '</p>',
                                  lambda x: '<p style="text-align: center;">' + x + '</p>'
                                 ]).replace('<th>', '<th style="text-align: center;">')


In [ ]:
print("resFile: ", resFile, "\n annFile: ", annFile)

In [ ]:
# Crear objeto coco y objeto cocoRes.
coco = COCO(annFile)
cocoRes = coco.loadRes(resFile)

# Crear objeto cocoEval.
cocoEval = COCOEvalCap(coco, cocoRes)

# Evaluar conjunto de imágenes.
cocoEval.params['image_id'] = cocoRes.getImgIds()

# Evaluar predicciones.
metrics = cocoEval.evaluate()

In [ ]:
# [OPCIONAL]: Guardar scores

scoreFile=savedDir+'score-'+savedFile+'.txt'

from contextlib import redirect_stdout
 
with open(scoreFile, 'w') as f:
    with redirect_stdout(f):
        print("File: "+resFile.split("/")[-1])
        for metric, score in cocoEval.eval.items():
            print('- %s: %.3f'%(metric, score))

!cat $scoreFile

In [ ]:
evals = [eva for eva in cocoEval.evalImgs if eva['CIDEr']<30]
num_imgs = 20
num_random = randint(0,len(cocoEval.params['image_id'])-num_imgs)

img_ids = pd.DataFrame(evals[num_random:num_random+num_imgs]).sort_values(['image_id'], ascending=[1])['image_id']

df = pd.DataFrame()

score_all = pd.DataFrame(columns=['image_id','CIDEr','SPICE'])

for i,imgId in enumerate(img_ids):
    
    annIds =  coco.getAnnIds(imgIds=imgId)
    anns = coco.loadAnns(annIds)
    row_df = pd.DataFrame(anns)
    row_df['caption'] = "R: " + row_df['caption'] + "<br>"
    df = pd.concat([row_df, df], ignore_index=True)

    annIds = cocoRes.getAnnIds(imgIds=imgId)
    anns = cocoRes.loadAnns(annIds)
   
    row_df = pd.DataFrame(anns)
    row_df['caption'] = "P: " + row_df['caption'] + "<br> <br>"
    df = pd.concat([row_df, df], ignore_index=True)
    
    indexes_all = pd.DataFrame(cocoEval.evalImgs)['image_id'].index
    index_img = indexes_all[pd.DataFrame(cocoEval.evalImgs)['image_id'] == imgId].tolist()[0]
    
    score_all.loc[i, 'image_id'] = imgId
    score_all.loc[i,'SPICE'] = pd.DataFrame(cocoEval.evalImgs)['SPICE'][index_img]['All']['f']
    score_all.loc[i,'CIDEr'] = pd.DataFrame(cocoEval.evalImgs)['CIDEr'][index_img]

captions_list = df[['image_id','caption']].groupby('image_id').sum().sort_values(['image_id'], ascending=[1])['caption'].tolist()
score_all_sorted = score_all.sort_values(['image_id'], ascending=[1])

In [ ]:
ms_coco_dir = '../ms-coco'
html = result_html(coco_dir= ms_coco_dir,
                image_ids=score_all_sorted['image_id'],
                image_captions=captions_list,
                score = score_all_sorted,
                image_scale=0.3)



In [ ]:
HTML(html)


In [ ]:
# [OPCIONAL]: Guardar HTML
HTMLFile=savedDir+'HTML-'+savedFile+'.html'
with open(HTMLFile, 'w') as file:
    file.write(html)

In [ ]:
# Plot score histogram
ciderScores = [eva['CIDEr'] for eva in cocoEval.evalImgs]
plt.hist(ciderScores)
plt.title('Histogram of CIDEr Scores', fontsize=20)
plt.xlabel('CIDEr score', fontsize=20)
plt.ylabel('result counts', fontsize=20)

# [OPCIONAL]: Guardar HISTOGRAM
HTMLFile=savedDir+'HISTOGRAM-'+savedFile+'.png'
plt.savefig(HTMLFile)